<a href="https://colab.research.google.com/github/astrodatos/proyecto_iluminatiuwu/blob/master/Algoritmo_(beta).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.io import fits #importamos fits aunque no creemos que lo usaremos
from astropy.wcs import WCS #importamos WCS aunque no creemos que lo usaremos
!pip install astroquery #esto para instalar astroquery
from astroquery.gaia import Gaia 

     |████████████████████████████████| 4.2MB 2.8MB/s 
     |████████████████████████████████| 2.3MB 33.4MB/s 
     |████████████████████████████████| 61kB 24.1MB/s 
     |████████████████████████████████| 102kB 30.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/8a/d1/1e/4124d9ef35e2bbfbaa284c97dd49d9babbab42d966c4bea190
Successfully built astroquery
Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [2]:
job = Gaia.launch_job_async("select top 1000000 \
solution_id,random_index,source_id,ra,dec,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,bp_rp,bp_g,g_rp,teff_val,lum_val \
from gaiadr2.gaia_source order by random_index") #obtenemos los datos de 1000000 estrellas con los anteriores atributos en un Asynchronous query

print(job) 
r = job.get_results()
print(r)

Query finished.
<Table masked=True length=1000000>
      name        dtype  unit format                            description                             n_bad 
---------------- ------- ---- ------ ------------------------------------------------------------------ ------
     solution_id   int64                                                            Solution Identifier      0
    random_index   int64                                            Random index used to select subsets      0
       source_id   int64             Unique source identifier (unique within a particular Data Release)      0
              ra float64  deg {!r:>}                                                    Right ascension      0
             dec float64  deg {!r:>}                                                        Declination      0
 phot_g_mean_mag float32  mag {!r:>}                                              G-band mean magnitude      0
phot_bp_mean_mag float32  mag {!r:>}                         

In [3]:
jobpd=r.to_pandas() #convertimos el job a pandas
jobpd[:10] #vemos las primeras 10 filas para ver que todo está bien

,solution_id,random_index,source_id,ra,dec,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,bp_rp,bp_g,g_rp,teff_val,lum_val
0,1635721458409799680,0,4205572950572486656,284.997397,-7.213794,18.246519,18.827530,17.534872,1.292658,0.581011,0.711647,NaN,NaN
1,1635721458409799680,1,4205565597589384320,285.403309,-7.148584,17.277395,17.743320,16.645607,1.097713,0.465925,0.631788,NaN,NaN
2,1635721458409799680,2,4053206649305182080,275.626769,-25.133812,19.644129,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1635721458409799680,3,4205568964799943680,285.346380,-7.110390,20.402403,20.705967,19.725151,0.980816,0.303564,0.677252,NaN,NaN
4,1635721458409799680,4,4049398731303340416,274.009409,-30.525125,19.241173,19.158228,18.095499,1.062729,-0.082945,1.145674,NaN,NaN
5,1635721458409799680,5,4053206924184153472,275.737952,-25.143911,19.238544,19.359943,17.496092,1.863852,0.121399,1.742453,NaN,NaN
6,1635721458409799680,6,2944217403015932032,96.875156,-15.996720,21.065298,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1635721458409799680,7,4205572263340546688,284.957509,-7.214939,20.697376,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1635721458409799680,8,5889059805439854848,234.271140,-52.218658,16.638042,17.331743,15.816595,1.515148,0.693701,0.821447,4368.459961,NaN
9,1635721458409799680,9,4049398765669294976,274.018619,-30.513203,19.936489,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
pd.Series.count(jobpd.iloc[:,11]) #con esto obtenemos el número de estrellas que poseen info de teff_val

95259

In [0]:
data={'random_index':jobpd.iloc[:,1],'class':np.zeros(1000000)}
wea=pd.DataFrame(data,columns=['random_index','class'])

In [0]:
mergeado=pd.merge(jobpd,wea,how='left',on='random_index')
for i in mergeado.iloc[:,1]:
  if mergeado.iloc[i,11]<=33000:
    mergeado.iloc[i,13]='O'
  if mergeado.iloc[i,11]>=10000 and mergeado.iloc[i,11]<30000:
    mergeado.iloc[i,13]='B'
  if mergeado.iloc[i,11]>=7500 and mergeado.iloc[i,11]<10000:
    mergeado.iloc[i,13]='A'
  if mergeado.iloc[i,11]>=6000 and mergeado.iloc[i,11]<7500:
    mergeado.iloc[:,13]='F'
  if mergeado.iloc[i,11]>=5200 and mergeado.iloc[i,11]<6000:
    mergeado.iloc[i,13]='G'
  if mergeado.iloc[i,11]>=3700 and mergeado.iloc[i,11]<5200:
    mergeado.iloc[i,13]='K'
  if mergeado.iloc[i,11]<=3700:
    mergeado.iloc[i,13]='M'
mergeado 

In [0]:
from sklearn.neural_network import MLPClassifier #por ahora usaremos estos algoritmos, pero desconocemos más o menos como procesar los datos,
from sklearn.ensemble import RandomForestClassifier  
#regresion logistica y sbmm